<a href="https://colab.research.google.com/github/johanjun/Google_Machine_Learning_Bootcamp_KR_2021/blob/main/kaggle/RSNA_MICCAI_BRAIN_TUMOR/rsna_miccai_brain_tumor_johanjun_1015_alldata_gridsearch(pre%2C_aug).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install pydicom

     |████████████████████████████████| 2.0 MB 5.2 MB/s 


In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from tqdm import tqdm
import random
import pydicom
import cv2
import pickle

from pydicom.pixel_data_handlers.util import apply_voi_lut

window_center = -500

import matplotlib.pyplot as plt
import glob

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [4]:
data_dir = '/content/drive/MyDrive/Google_Machine_Learning_Bootcamp_KR_2021/RSNA_MICCAI/data/rsna-miccai/'

In [5]:
train_df = pd.read_csv(data_dir + "train_labels.csv")
test_df = pd.read_csv(data_dir + "sample_submission.csv")

In this discussion a competition host has notified that there are some issues with these 3 cases
Patient IDs -

1. 00109 (FLAIR images are blank)
2. 00123 (T1w images are blank)
3. 00709 (FLAIR images are blank)
<br><br>Hence these can be excluded

In [6]:
#refer: https://www.kaggle.com/arnabs007/part-1-rsna-miccai-btrc-understanding-the-data
EXCLUDE = [109, 123, 709]
train_df = train_df[~train_df.BraTS21ID.isin(EXCLUDE)]

In [7]:
TYPES = ["FLAIR", "T1w", "T1wCE", "T2w"] #mpMRI scans

In [8]:
def load_dicom(path, size = 128): #load DICOM files
    dicom = pydicom.read_file(path)
    s = int(dicom.RescaleSlope)
    b = int(dicom.RescaleIntercept)
    data = s * dicom.pixel_array + b

    dicom.WindowCenter = window_center
    dicom.WindowWidth = window_width

    data = apply_voi_lut(data, dicom)

    data = np.clip(data, window_center - (window_width / 2), window_center + (window_width / 2))

    data = dicom.pixel_array #returns a numpy.ndarray containing the pixel data
    if np.max(data) != 0:
        data = data / np.max(data) #standardizes so that the pixel values are between 0 and 1
    data = (data * 255).astype(np.uint8) #rescales to 0 and 255
    return cv2.resize(data, (size, size))

In [9]:
def get_all_image_paths(BraTS21ID, image_type, folder="train"): #get an array of all the images of a particular type or a particular patient id
    assert(image_type in TYPES) #only in types
    patient_path = os.path.join(data_dir + "%s/" % folder, str(BraTS21ID).zfill(5)) #다른 폴더일 수도 있음
    #print(lambda x: int(x[:-4].split("-")[-1]))
    
    paths = sorted(glob.glob(os.path.join(patient_path, image_type, "*")), key=lambda x: int(x[:-4].split("-")[-1])) #sort
    #print(paths)
    
    num_images = len(paths)
    
    start = int(num_images * 0.25)
    end = int(num_images * 0.75)
    if num_images < 10:
        jump = 1
    else:
        jump = 3
        
    return np.array(paths[start:end:jump])

In [10]:
def get_all_images(BraTS21ID, image_type, folder="train", size=225):
    return [load_dicom(path, size) for path in get_all_image_paths(BraTS21ID, image_type, folder)]

In [11]:
def get_all_data_train(image_type):
    global train_df
    
    X = []
    y = []
    train_ids = []

    for i in tqdm(train_df.index):
        tmp_x = train_df.loc[i]
        images = get_all_images(int(tmp_x["BraTS21ID"]), image_type, "train", IMAGE_SIZE)
        label = tmp_x["MGMT_value"]

        X += images
        y += [label] * len(images)
        train_ids += [int(tmp_x["BraTS21ID"])] * len(images)
        assert(len(X) == len(y))
    return np.array(X), np.array(y), np.array(train_ids)

def get_all_data_test(image_type):
    global test_df
    
    X = []
    test_ids = []

    for i in tqdm(test_df.index):
        tmp_x = test_df.loc[i]
        images = get_all_images(int(tmp_x["BraTS21ID"]), image_type, "test", IMAGE_SIZE)
        X += images
        test_ids += [int(tmp_x["BraTS21ID"])] * len(images)

    return np.array(X), np.array(test_ids)

In [12]:
#pickling
with open(data_dir+'X_T1wCE.pkl', 'rb') as f:
  X_T1wCE = pickle.load(f)

with open(data_dir+'y_T1wCE.pkl', 'rb') as f:
  y_T1wCE = pickle.load(f)

with open(data_dir+'train_idt_T1wCE.pkl', 'rb') as f:
  train_idt_T1wCE = pickle.load(f)

with open(data_dir+'X_test_T1wCE.pkl', 'rb') as f:
  X_test_T1wCE = pickle.load(f)

with open(data_dir+'test_idt_T1wCE.pkl', 'rb') as f:
  test_idt_T1wCE = pickle.load(f)
#pickling
with open(data_dir+'X_FLAIR.pkl', 'rb') as f:
  X_FLAIR = pickle.load(f)

with open(data_dir+'y_FLAIR.pkl', 'rb') as f:
  y_FLAIR = pickle.load(f)

with open(data_dir+'train_idt_FLAIR.pkl', 'rb') as f:
  train_idt_FLAIR = pickle.load(f)

with open(data_dir+'X_test_FLAIR.pkl', 'rb') as f:
  X_test_FLAIR = pickle.load(f)

with open(data_dir+'test_idt_FLAIR.pkl', 'rb') as f:
  test_idt_FLAIR = pickle.load(f)
#pickling
with open(data_dir+'X_T2w.pkl', 'rb') as f:
  X_T2w = pickle.load(f)

with open(data_dir+'y_T2w.pkl', 'rb') as f:
  y_T2w = pickle.load(f)

with open(data_dir+'train_idt_T2w.pkl', 'rb') as f:
  train_idt_T2w = pickle.load(f)

with open(data_dir+'X_test_T2w.pkl', 'rb') as f:
  X_test_T2w = pickle.load(f)

with open(data_dir+'test_idt_T2w.pkl', 'rb') as f:
  test_idt_T2w = pickle.load(f)
#pickling
with open(data_dir+'X_T1w.pkl', 'rb') as f:
  X_T1w = pickle.load(f)

with open(data_dir+'y_T1w.pkl', 'rb') as f:
  y_T1w = pickle.load(f)

with open(data_dir+'train_idt_T1w.pkl', 'rb') as f:
  train_idt_T1w = pickle.load(f)

with open(data_dir+'X_test_T1w.pkl', 'rb') as f:
  X_test_T1w = pickle.load(f)

with open(data_dir+'test_idt_T1w.pkl', 'rb') as f:
  test_idt_T1w = pickle.load(f)

In [13]:
X = np.concatenate([X_T1wCE, X_T2w, X_T1w, X_FLAIR], axis=0)
y = np.concatenate([y_T1wCE, y_T2w, y_T1w, y_FLAIR], axis=0)
train_idt = np.concatenate([train_idt_T1wCE, train_idt_T2w, train_idt_T1w, train_idt_FLAIR], axis=0)

X_test = np.concatenate([X_test_FLAIR, X_test_T1wCE, X_test_T2w, X_test_T1w], axis=0)
test_idt = np.concatenate([test_idt_FLAIR, test_idt_T1wCE, test_idt_T2w, test_idt_T1w], axis=0)

In [14]:
from tensorflow.keras import layers

data_augmentation = tf.keras.Sequential([
  layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
  layers.experimental.preprocessing.RandomRotation(0.2),
])

In [21]:
X_train, X_valid, y_train, y_valid, train_idt_train, train_idt_valid = train_test_split(X, y, train_idt, test_size=0.1, random_state=42)

X_train_exp = tf.expand_dims(X_train, -1)
augmented_image = data_augmentation(X_train_exp)
X_train_sq = tf.squeeze(augmented_image)

X_train = np.concatenate([X_train, X_train_sq], axis=0)
y_train = np.concatenate([y_train, y_train], axis=0)

train_idt_train = np.concatenate([train_idt_train, train_idt_train], axis=0)

In [22]:
X_train = tf.expand_dims(X_train, axis=-1) #expand the dimension at the end of the array
X_valid = tf.expand_dims(X_valid, axis=-1)
X_test = tf.expand_dims(X_test, axis=-1)

y_train = to_categorical(y_train) #one-hot incoding
y_valid = to_categorical(y_valid)

In [23]:
X_train.shape, y_train.shape, X_valid.shape, y_valid.shape, train_idt_train.shape, train_idt_valid.shape

(TensorShape([105256, 128, 128, 1]),
 (105256, 2),
 TensorShape([5848, 128, 128, 1]),
 (5848, 2),
 (105256,),
 (5848,))

In [24]:
X2 = np.concatenate([X_FLAIR, X_T2w, X_T1w, X_T1wCE], axis=0)
y2 = np.concatenate([y_FLAIR, y_T2w, y_T1w, y_T1wCE], axis=0)
train_idt2 = np.concatenate([train_idt_FLAIR, train_idt_T2w, train_idt_T1w, train_idt_T1wCE], axis=0)

In [25]:
X_train2, X_valid2, y_train2, y_valid2, train_idt_train2, train_idt_valid2 = train_test_split(X2, y2, train_idt2, test_size=0.05)

In [26]:
X_train2 = np.expand_dims(X_train2, axis=-1) 
X_valid2 = np.expand_dims(X_valid2, axis=-1)

y_train2 = to_categorical(y_train2) 
y_valid2 = to_categorical(y_valid2)

In [27]:
# %load train_lib.py

def train():
    import wandb
    config_defaults = {
        # 'N_EPOCHS' : 15,
        'N_BATCH' : 32,
        'DENSE' : 128,
        'DROUPOUT' : 0.1
    }
    wandb.init(project='RSNA_1015_alldata_augmented',
               config=config_defaults,
               magic=True)

    config = wandb.config
    

    IMAGE_SIZE = 128
    N_EPOCHS = 40
    N_BATCH = config.N_BATCH
    # N_BATCH = 32
    N_TRAIN = X_train.shape[0]
    N_VAL = X_valid.shape[0]
    steps_per_epoch = N_TRAIN / N_BATCH
    validation_steps = int(np.ceil(N_VAL / N_BATCH))

    def create_model():
      model = tf.keras.Sequential()
      model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', padding='VALID', 
                                    input_shape=(IMAGE_SIZE, IMAGE_SIZE, 1)))
      model.add(tf.keras.layers.BatchNormalization())
      model.add(tf.keras.layers.MaxPool2D(padding='SAME'))
      
      model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu', padding='SAME'))
      model.add(tf.keras.layers.BatchNormalization())
      model.add(tf.keras.layers.MaxPool2D(padding='SAME'))

      model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu', padding='SAME'))
      model.add(tf.keras.layers.BatchNormalization())
      model.add(tf.keras.layers.MaxPool2D(padding='SAME'))

      model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu', padding='VALID'))
      model.add(tf.keras.layers.BatchNormalization())
      model.add(tf.keras.layers.MaxPool2D(padding='SAME'))
      

      model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu', padding='SAME'))
      model.add(tf.keras.layers.BatchNormalization())
      model.add(tf.keras.layers.MaxPool2D(padding='SAME'))
      model.add(tf.keras.layers.Dropout(config.DROPOUT))

      model.add(tf.keras.layers.Flatten())
      model.add(tf.keras.layers.Dense(config.DENSE, activation='relu'))
      model.add(tf.keras.layers.Dense(2, activation='softmax'))
      return model
    
    model = create_model()
    

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=[tf.keras.metrics.AUC()])
    
    model.fit(
    x=X_train,
    y=y_train,
    epochs=N_EPOCHS,
    steps_per_epoch=steps_per_epoch,
    validation_data=(X_valid2, y_valid2))

In [28]:
sweep_config = {
    'method': 'grid',
    'parameters': {
        # 'N_EPOCHS': {
        #     'values': [40, 65, 90, 150]
        # },
        'N_BATCH': {
            'values': [32, 128]
        },
        'DENSE': {
            'values': [128, 512]
        },
        'DROPOUT': {
            'values': [0.1, 0.4]
        },
    }
}

In [29]:
%pip install -q wandb

     |████████████████████████████████| 1.7 MB 6.6 MB/s 
     |████████████████████████████████| 180 kB 52.6 MB/s 
     |████████████████████████████████| 139 kB 83.7 MB/s 
     |████████████████████████████████| 97 kB 7.2 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 


In [30]:
!wandb login

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [32]:
import wandb
sweep_id = wandb.sweep(sweep_config, project='RSNA_1015_alldata_augmented')

Create sweep with ID: 04x235h9
Sweep URL: https://wandb.ai/johanjunme/RSNA_1015_alldata_augmented/sweeps/04x235h9


In [33]:
wandb.agent(sweep_id, function=train)

wandb: Agent Starting Run: hun4cv8m with config:
wandb: 	DENSE: 128
wandb: 	DROPOUT: 0.1
wandb: 	N_BATCH: 32
wandb: Currently logged in as: johanjunme (use `wandb login --relogin` to force relogin)
wandb: WARNING Ignored wandb.init() arg project when running a sweep


wandb: WARNING Ignored wandb.init() arg project when running a sweep


Epoch 1/40
3289/3289 [==============================] - 62s 14ms/step - loss: 0.6833 - auc: 0.6015 - val_loss: 0.6692 - val_auc: 0.6284
Epoch 2/40
3289/3289 [==============================] - 45s 14ms/step - loss: 0.6711 - auc: 0.6184 - val_loss: 0.6536 - val_auc: 0.6544
Epoch 3/40
3289/3289 [==============================] - 45s 14ms/step - loss: 0.6595 - auc: 0.6402 - val_loss: 0.6353 - val_auc: 0.6847
Epoch 4/40
3289/3289 [==============================] - 45s 14ms/step - loss: 0.6355 - auc: 0.6752 - val_loss: 0.5752 - val_auc: 0.7498
Epoch 5/40
3289/3289 [==============================] - 44s 14ms/step - loss: 0.6047 - auc: 0.7136 - val_loss: 0.4947 - val_auc: 0.8332
Epoch 6/40
3289/3289 [==============================] - 45s 14ms/step - loss: 0.5743 - auc: 0.7479 - val_loss: 0.4455 - val_auc: 0.8754
Epoch 7/40
3289/3289 [==============================] - 45s 14ms/step - loss: 0.5461 - auc: 0.7774 - val_loss: 0.4129 - val_auc: 0.8888
Epoch 8/40
3289/3289 [==========================

auc,▁▁▂▂▃▄▄▅▅▅▆▆▆▇▇▇▇▇▇▇▇███████████████████
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,███▇▇▇▆▆▆▅▅▅▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
val_auc,▁▁▂▃▅▆▆▆▇▇▇▇████████████████████████████
val_loss,███▇▆▅▅▅▄▄▃▃▃▂▂▂▂▂▂▂▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁
auc,0.99282
best_epoch,28
best_val_loss,0.08841
epoch,39
loss,0.10573
val_auc,0.99265


wandb: Agent Starting Run: 4ve52fep with config:
wandb: 	DENSE: 128
wandb: 	DROPOUT: 0.1
wandb: 	N_BATCH: 128
wandb: WARNING Ignored wandb.init() arg project when running a sweep


...

auc,▁▁▁▂▂▃▃▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇████████████
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,████▇▇▇▆▆▆▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁
val_auc,▁▁▁▂▃▄▅▅▆▆▇▇▇▇▇█████████████████████████
val_loss,████▇▆▆▆▅▅▅▄▄▃▃▃▃▂▂▂▂▂▂▂▂▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁
auc,0.97199
best_epoch,37
best_val_loss,0.09048
epoch,39
loss,0.21359
val_auc,0.99222


wandb: Agent Starting Run: 3f0bfgni with config:
wandb: 	DENSE: 512
wandb: 	DROPOUT: 0.4
wandb: 	N_BATCH: 128
wandb: WARNING Ignored wandb.init() arg project when running a sweep


Epoch 1/40
822/822 [==============================] - 35s 41ms/step - loss: 0.7052 - auc: 0.5924 - val_loss: 0.6716 - val_auc: 0.6300
Epoch 2/40
822/822 [==============================] - 33s 40ms/step - loss: 0.6732 - auc: 0.6130 - val_loss: 0.6628 - val_auc: 0.6442
Epoch 3/40
822/822 [==============================] - 33s 41ms/step - loss: 0.6667 - auc: 0.6269 - val_loss: 0.6301 - val_auc: 0.6902
Epoch 4/40
822/822 [==============================] - 33s 40ms/step - loss: 0.6554 - auc: 0.6458 - val_loss: 0.6112 - val_auc: 0.7188
Epoch 5/40
822/822 [==============================] - 33s 40ms/step - loss: 0.6424 - auc: 0.6660 - val_loss: 0.5791 - val_auc: 0.7487
Epoch 6/40
822/822 [==============================] - 33s 41ms/step - loss: 0.6237 - auc: 0.6909 - val_loss: 0.5488 - val_auc: 0.7817
Epoch 7/40
822/822 [==============================] - 33s 40ms/step - loss: 0.6020 - auc: 0.7179 - val_loss: 0.5168 - val_auc: 0.8114
Epoch 8/40
822/822 [==============================] - 33s 40ms

auc,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇████████████
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,██▇▇▇▇▇▆▆▆▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁
val_auc,▁▁▂▃▃▄▄▅▅▆▇▇▇▇▇█████████████████████████
val_loss,███▇▇▇▆▆▅▅▄▄▃▃▃▃▂▃▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁
auc,0.97714
best_epoch,37
best_val_loss,0.07316
epoch,39
loss,0.19294
val_auc,0.99514


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
